## Importing required modules.
### Numpy modules module required to perform basic matrix operations.

In [5]:
import numpy as np

### Bezier extraction operator.

In [10]:
def bezier_extraction(knot,p):
    """
    Input: 
        Knot --> Knot vector
        p    --> Degree
    
    Output:
        C  --> 3 dimentional Extraction matrix (numpy array). 
               Each p+1,p+1 matrix is the local extraction operator.
        ne --> number of local extraction matrices.
        
    Test:
        knot = [0,0,0,0.5,1,1,1]
        p = 2
        [C, ne] = bezierExtraction(knot,p)
    """


    m  = len(knot)-p-1
    a  = p+1
    b  = a+1
    ne = 0
    C = []
    C.append(np.eye(p+1,p+1))
    alphas = {}
    #numerator = []
    
    while b <= m:
        C.append(np.eye(p+1,p+1))
        i=b
        while b <= m and knot[b] == knot[b-1]:
            b=b+1
            
        multiplicity = b-i+1
        if multiplicity < p:
            numerator = (knot[b-1]-knot[a-1])
            for j in range(p,multiplicity,-1):
                alphas[j-multiplicity]=numerator/(knot[a+j-1]-knot[a-1])

            r=p-multiplicity
            for j in range(1,r+1):
                save = r-j+1
                s = multiplicity + j
                for k in range(p+1,s,-1):
                    alpha=alphas[k-s]
                    C[ne][:,k-1]= alpha*C[ne][:,k-1] + (1-alpha)*C[ne][:,k-2]
                if b <= m:
                    C[ne+1][save-1:save+j, save-1] = C[ne][p-j:p+1, p]
            ne=ne+1
            if b <= m:
                a=b
                b=b+1

        elif multiplicity == p:
            if b <= m:
                ne=ne+1
                a=b
                b=b+1
    return C, ne+1


In [12]:
knot = [0,0,0,0.5,1,1,1]
p=2
[C, ne] = bezier_extraction(knot,p)

## Number of extraction matrices
### ne = 2

In [18]:
ne

2

## First local extraction operator
$\textrm{C}[0] = \begin{bmatrix} 1.0 & 0.0 & 0.0 \\ 0.0 & 1.0 & 0.5 \\ 0.0 & 0.0 & 0.5 \\\end{bmatrix}$

In [19]:
C[0]

array([[1. , 0. , 0. ],
       [0. , 1. , 0.5],
       [0. , 0. , 0.5]])

# Second Extraction operator
$\textrm{C}[1] = \begin{bmatrix} 0.5 & 0.0 & 0.0 \\ 0.5 & 1.0 & 0.0 \\ 0.0 & 0.0 & 1.0 \end{bmatrix}$

In [20]:
C[1]

array([[0.5, 0. , 0. ],
       [0.5, 1. , 0. ],
       [0. , 0. , 1. ]])

2